# Nutree Tutorial

Nutree organizes arbitrary object instances in an unobtrusive way. <br>
That means, we can add existing objects without havin to derrive from a common 
base class or having them implement a specific protocol.

# Setup some sample classes and objects

In [31]:
import uuid


class Department:
    def __init__(self, name: str):
        self.guid = uuid.uuid4()
        self.name = name

    def __str__(self):
        return f"Department<{self.name}>"


class Person:
    def __init__(self, name: str, age: int):
        self.guid = uuid.uuid4()
        self.name = name
        self.age = age

    def __str__(self):
        return f"Person<{self.name} ({self.age})>"

Now create some instances

In [32]:
development_dep = Department("Development")
test__dep = Department("Test")
marketing_dep = Department("Marketing")

alice = Person("Alice", 25)
bob = Person("Bob", 35)
claire = Person("Claire", 45)
dave = Person("Dave", 55)

Let's organize these objects in a hierarchical structure:

In [33]:
from nutree import Tree

tree = Tree("Organization")

dev_node = tree.add(development_dep)
test_node = dev_node.add(test__dep)
mkt_node = tree.add(marketing_dep)

tree.add(alice)
dev_node.add(bob)
test_node.add(claire)
mkt_node.add(dave)

tree.print()

Tree<'Organization'>
├── <__main__.Department object at 0x111f04e00>
│   ├── <__main__.Department object at 0x111b89f70>
│   │   ╰── <__main__.Person object at 0x111f05520>
│   ╰── <__main__.Person object at 0x111f04da0>
├── <__main__.Department object at 0x111edac90>
│   ╰── <__main__.Person object at 0x111f06a50>
╰── <__main__.Person object at 0x111ed9880>


Tree nodes store a reference to the object in the `node.data` attribute.

The nodes are formatted by the object's  `__repr__` implementation by default. <br>
We can overide ths by passing an f-string as `repr` argument:

In [34]:
tree.print(repr="{node.data}")

Tree<'Organization'>
├── Department<Development>
│   ├── Department<Test>
│   │   ╰── Person<Claire (45)>
│   ╰── Person<Bob (35)>
├── Department<Marketing>
│   ╰── Person<Dave (55)>
╰── Person<Alice (25)>


# Iteration and Searching

# Mutation

# Data IDs and Clones

In [35]:
tree.print(repr="{node}", title=False)

Node<'Department<Development>', data_id=287245536>
├── Node<'Department<Test>', data_id=287017463>
│   ╰── Node<'Person<Claire (45)>', data_id=287245650>
╰── Node<'Person<Bob (35)>', data_id=287245530>
Node<'Department<Marketing>', data_id=287234761>
╰── Node<'Person<Dave (55)>', data_id=287245989>
Node<'Person<Alice (25)>', data_id=287234440>


# Serialization

In [36]:
tree.to_dict_list()

[{'data': 'Department<Development>',
  'children': [{'data': 'Department<Test>',
    'children': [{'data': 'Person<Claire (45)>'}]},
   {'data': 'Person<Bob (35)>'}]},
 {'data': 'Department<Marketing>',
  'children': [{'data': 'Person<Dave (55)>'}]},
 {'data': 'Person<Alice (25)>'}]

In [37]:
list(tree.to_list_iter())

[(0, {}), (1, {}), (2, {}), (1, {}), (0, {}), (5, {}), (0, {})]

In [38]:
t = Tree._from_list([(0, "A")])
print(t.format())

Tree<'4595934048'>
╰── 'A'
